In [ ]:
import pandas as pd
import pandas_datareader.data as pdr
from datetime import datetime
from datetime import date
from datetime import timedelta
import math
import yfinance as yf

In [ ]:
# Step 1: Get and organize data
# Step 2: Run algorithm
# Step 3: Calculate/Analyze Return vs Benchmark(s)
# Step 4: Test Inputs/Tweak Strategy

In [ ]:
def cagr(_df):
    df = _df.copy()
    df = df.sort_index(ascending=True)
    df["cum_return"] = (1 + df["return"]).cumprod()
    total_ret = df['cum_return'][-1]
    start = datetime.utcfromtimestamp(df.index.values[0].astype('O')/1e9)
    end = datetime.utcfromtimestamp(df.index.values[-1].astype('O')/1e9)
    period_years = (end - start).days / 365.25
    CAGR = (total_ret)**(1/period_years) - 1
    return CAGR

In [ ]:
def volatility(_df):
    start = datetime.utcfromtimestamp(_df.index.values[0].astype('O')/1e9)
    end = datetime.utcfromtimestamp(_df.index.values[-1].astype('O')/1e9)
    period_years = (end - start).days / 365.25
    ratio_to_annual = _df['return'].count() / period_years
    vol = _df["return"].std() * math.sqrt(ratio_to_annual)
    return vol

In [ ]:
def sharpe(_df, risk_free_rate):
    ret = cagr(_df)
    vol = volatility(_df)
    sharpe = (ret - risk_free_rate) / vol
    return sharpe

In [ ]:
def max_drawdown(_df):
    df = _df.copy()
    df = df.sort_index(ascending=True)
    df["cum_return"] = (1 + df["return"]).cumprod()
    df["cum_return_max"] = df["cum_return"].cummax()
    df["drawdown"] = df["cum_return_max"] - df["cum_return"]
    df["drawdown_pct"] = df["drawdown"] / df["cum_return_max"]
    max_dd = df.loc[df["drawdown_pct"].idxmax()]
    return {
        'drawdown': max_dd['drawdown_pct'],
        'date': max_dd.name
    }

In [ ]:
# Step 1: Get and organize data

sp500_ticker = yf.Ticker('^GSPC')
input_df = sp500_ticker.history(period="max")[['Close']]
input_df['1 Day Ret'] = input_df['Close'].pct_change()
input_df['1 Day Prev Chng'] = input_df['Close'].pct_change().shift(1)
input_df['5 Day Prev Chng'] = input_df['Close'].pct_change(5).shift(1)
input_df['30 Day Prev Chng'] = input_df['Close'].pct_change(30).shift(1)
input_df[:40]

In [ ]:
# Step 2: Run algorithm

def my_strategy(_df, gt_5):
    df = _df.copy()

    for x in df.index:  
        row = df.loc[x]
        if(row['5 Day Prev Chng'] > gt_5):
            df.at[x,'return'] = 0
        else:
            df.at[x,'return'] = row['1 Day Ret']

    return df

In [ ]:
strategy_returns = my_strategy(input_df, 0.08)

In [ ]:
sharpe(strategy_returns, 0.02)

In [ ]:
cagr(strategy_returns)

In [ ]:
max_drawdown(strategy_returns)

In [ ]:
# Step 3: Calculate Benchmark(s)

sp500 = sp500_ticker.history(period="max")[['Close']]
sp500['return'] = sp500['Close'].pct_change()
sharpe(sp500, 0.02)

In [ ]:
cagr(sp500)

In [ ]:
max_drawdown(sp500)

In [ ]:
# Step 4: Test Inputs/Tweak Strategy

In [ ]:
max_result = 0
max_result_input = None
result_df = pd.DataFrame(columns=('input', 'result'))

for x in range(0, 100):
    i = x / 500.0 - 0.1
    strategy_returns = my_strategy(input_df, i)
    result = cagr(strategy_returns)
    print("input: {:.4f} = {:.4f}".format(i, result))
    result_df.loc[x] = [i, result]
    if (result > max_result):
        max_result = result
        max_result_input = i
    
print("max_result: ", max_result)
print("max_result_input: ", max_result_input)

In [ ]:
result_df

In [ ]:
result_df = result_df.set_index('input')

In [ ]:
result_df['result'].plot(grid=True)